# Running attribute inference attacks on the Nursery data

In this tutorial we will show how to run both black-box and white-box inference attacks. This will be demonstrated on the Nursery dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/datasets/nursery). 

## Preliminaries
In order to mount a successful attribute inference attack, the attacked feature must be categorical, and with a relatively small number of possible values (preferably binary, but should at least be less then the number of label classes).

In the case of the nursery dataset, the sensitive feature we want to infer is the 'social' feature. In the original dataset this is a categorical feature with 3 possible values. To make the attack more successful, we reduced this to two possible feature values by assigning the original value 'problematic' the new value 1, and the other original values were assigned the new value 0.

We have also already preprocessed the dataset such that all categorical features are one-hot encoded, and the data was scaled.

## Load data

In [1]:
import pandas as pd

df = pd.read_csv('../utils/data/inference/Nursery_social_prepared_train.csv', sep=',', engine='python')

df

,label,children,social,parents_pretentious,parents_great_pret,parents_usual,has_nurs_very_crit,has_nurs_improper,has_nurs_proper,has_nurs_critical,...,form_incomplete,form_foster,housing_critical,housing_convenient,housing_less_conv,finance_convenient,finance_inconv,health_priority,health_recommended,health_not_recom
0,1,0.444450,-0.704142,1.434509,-0.713050,-0.711204,2.000724,-0.499216,-0.500723,-0.505541,...,-0.567324,-0.577425,1.428869,-0.711511,-0.709974,0.985252,-0.985252,1.399405,-0.708745,-0.698019
1,1,0.444450,-0.704142,-0.697102,1.402427,-0.711204,-0.499819,2.003141,-0.500723,-0.505541,...,-0.567324,-0.577425,-0.699854,1.405459,-0.709974,-1.014968,1.014968,1.399405,-0.708745,-0.698019
2,3,1.335242,1.420169,1.434509,-0.713050,-0.711204,-0.499819,-0.499216,1.997111,-0.505541,...,-0.567324,-0.577425,-0.699854,-0.711511,1.408503,-1.014968,1.014968,-0.714590,1.410946,-0.698019
3,3,0.444450,-0.704142,-0.697102,-0.713050,1.406067,-0.499819,-0.499216,-0.500723,1.978079,...,1.762661,-0.577425,-0.699854,1.405459,-0.709974,0.985252,-0.985252,1.399405,-0.708745,-0.698019
4,3,0.444450,-0.704142,-0.697102,-0.713050,1.406067,-0.499819,-0.499216,1.997111,-0.505541,...,-0.567324,-0.577425,1.428869,-0.711511,-0.709974,0.985252,-0.985252,1.399405,-0.708745,-0.698019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,0,-1.337132,-0.704142,1.434509,-0.713050,-0.711204,-0.499819,-0.499216,-0.500723,1.978079,...,-0.567324,-0.577425,-0.699854,-0.711511,1.408503,-1.014968,1.014968,-0.714590,-0.708745,1.432625
5179,1,-1.337132,1.420169,-0.697102,1.402427,-0.711204,-0.499819,2.003141,-0.500723,-0.505541,...,1.762661,-0.577425,-0.699854,1.405459,-0.709974,-1.014968,1.014968,-0.714590,1.410946,-0.698019
5180,3,-0.446341,1.420169,-0.697102,-0.713050,1.406067,-0.499819,2.003141,-0.500723,-0.505541,...,1.762661,-0.577425,-0.699854,1.405459,-0.709974,-1.014968,1.014968,-0.714590,1.410946,-0.698019
5181,1,-0.446341,-0.704142,-0.697102,-0.713050,1.406067,2.000724,-0.499216,-0.500723,-0.505541,...,-0.567324,-0.577425,1.428869,-0.711511,-0.709974,0.985252,-0.985252,1.399405,-0.708745,-0.698019


## Train decision tree model

In [2]:
from sklearn.tree import DecisionTreeClassifier

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier

features = df.drop(['label'], axis=1)
labels = df.loc[:, 'label']
model = DecisionTreeClassifier()
model.fit(features, labels)

art_classifier = ScikitlearnDecisionTreeClassifier(model)

df_test = pd.read_csv('../utils/data/inference/Nursery_social_prepared_test.csv', sep=',', engine='python')
features_test = df_test.drop(['label'], axis=1)
labels_test = df_test.loc[:, 'label']
test_data = features_test.to_numpy()

print('Base model accuracy: ', model.score(features_test, labels_test))

Base model accuracy:  0.6851851851851852


## Attack
### Black-box attack
The black-box attack basically trains an additional classifier (called the attack model) to predict the attacked feature's value from the remaining n-1 features as well as the original (attacked) model's predictions.
#### Train attack model

In [12]:
import numpy as np
from art.attacks.inference import AttributeInferenceBlackBox

attack_feature = 1
data = features.to_numpy()

# training data without attacked feature
x_train_for_attack = np.delete(data, attack_feature, 1)
# only attacked feature
x_train_feature = data[:, attack_feature].copy().reshape(-1, 1)

bb_attack = AttributeInferenceBlackBox(art_classifier, attack_feature=attack_feature)

# get original model's predictions
x_train_predictions = np.array([np.argmax(arr) for arr in art_classifier.predict(data)]).reshape(-1,1)

# train attack model
bb_attack.fit(test_data)

#### Infer sensitive feature and check accuracy

In [13]:
# get inferred values
values = [-0.704141531, 1.420169037]
inferred_train_bb = bb_attack.infer(x_train_for_attack, x_train_predictions, values=values)
# check accuracy
train_acc = np.sum(inferred_train_bb == x_train_feature.reshape(1,-1)) / len(inferred_train_bb)
print(train_acc)

0.6158595408064828


This means that for 61% of the training set, the attacked feature is inferred correctly using this attack.

## Whitebox attacks
These two attacks do not train any additional model, they simply use additional information coded within the attacked decision tree model to compute the probability of each value of the attacked feature and outputs the value with the highest probability.
### First attack

In [14]:
from art.attacks.inference import AttributeInferenceWhiteBoxLifestyleDecisionTree

wb_attack = AttributeInferenceWhiteBoxLifestyleDecisionTree(art_classifier, attack_feature=attack_feature)

priors = [3465 / 5183, 1718 / 5183]

# get inferred values
inferred_train_wb1 = wb_attack.infer(x_train_for_attack, x_train_predictions, values=values, priors=priors)

# check accuracy
train_acc = np.sum(inferred_train_wb1 == x_train_feature.reshape(1,-1)) / len(inferred_train_wb1)
print(train_acc)

0.6183677406907196


### Second attack

In [15]:
from art.attacks.inference import AttributeInferenceWhiteBoxDecisionTree

wb2_attack = AttributeInferenceWhiteBoxDecisionTree(art_classifier, attack_feature=attack_feature)

# get inferred values
inferred_train_wb2 = wb2_attack.infer(x_train_for_attack, x_train_predictions, values=values, priors=priors)

# check accuracy
train_acc = np.sum(inferred_train_wb2 == x_train_feature.reshape(1,-1)) / len(inferred_train_wb2)
print(train_acc)

0.6905267219756898


The white-box attacks are able to correctly infer the attacked feature value in 61% and 69% of the test set respectively. 

Now let's check the precision and recall:

In [18]:
def calc_precision_recall(predicted, actual, positive_value=1):
    score = 0  # both predicted and actual are positive
    num_positive_predicted = 0  # predicted positive
    num_positive_actual = 0  # actual positive
    for i in range(len(predicted)):
        if predicted[i] == positive_value:
            num_positive_predicted += 1
        if actual[i] == positive_value:
            num_positive_actual += 1
        if predicted[i] == actual[i]:
            if predicted[i] == positive_value:
                score += 1
    
    if num_positive_predicted == 0:
        precision = 1
    else:
        precision = score / num_positive_predicted  # the fraction of predicted “Yes” responses that are correct
    if num_positive_actual == 0:
        recall = 1
    else:
        recall = score / num_positive_actual  # the fraction of “Yes” responses that are predicted correctly

    return precision, recall
    
# black-box
print(calc_precision_recall(inferred_train_bb, x_train_feature, positive_value=1.420169037))
# white-box 1
print(calc_precision_recall(inferred_train_wb1, x_train_feature, positive_value=1.420169037))
# white-box 2
print(calc_precision_recall(inferred_train_wb2, x_train_feature, positive_value=1.420169037))

(0.394919168591224, 0.29860302677532014)
(0.3402948402948403, 0.1612339930151339)
(0.5874233128834356, 0.2229336437718277)
